# keras.Sequential
- model.trainable_variables
- model.call()
# keras.layers.Layer
- \__init__
- call

# keras.Model
- \__init__
- call
- Model:compile/fit/evaluate/predict

- os.environ\["TF_CPP_MIN_LOG_LEVEL"]的取值有四个：0，1，2，3，分别和log的四个等级挂钩：INFO，WARNING，ERROR，FATAL（重要性由左到右递增）

- 当os.environ\["TF_CPP_MIN_LOG_LEVEL"]='0'的时候，输出信息：INFO + WARNING + ERROR + FATAL

- 当os.environ\["TF_CPP_MIN_LOG_LEVEL"]='1'的时候，输出信息：WARNING + ERROR + FATAL

- 当os.environ\["TF_CPP_MIN_LOG_LEVEL"]='2'的时候，输出信息：ERROR + FATAL

- 当os.environ\["TF_CPP_MIN_LOG_LEVEL"]='3'的时候，输出信息：FATAL

In [1]:
import tensorflow as tf 
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow import keras
import os

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'

In [3]:
def preprocess(x, y):
    # [0-255]  - [-1, 1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [4]:
batchsz = 128
(x, y), (x_val, y_val) = datasets.cifar10.load_data()

In [5]:
# [10k, 1]  [10k]  [10k , 10]
y = tf.squeeze(y)
y_val = tf.squeeze(y_val)
y = tf.one_hot(y, depth=10)
y_val = tf.one_hot(y_val, depth=10)
x.shape, y.shape, x_val.shape, y_val.shape, x.min(), x.max()

((50000, 32, 32, 3),
 TensorShape([50000, 10]),
 (10000, 32, 32, 3),
 TensorShape([10000, 10]),
 0,
 255)

In [6]:
train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batchsz)

In [7]:
sample = next(iter(train_db))
print(sample[0].shape, sample[1].shape)

(128, 32, 32, 3) (128, 10)


In [8]:
class MyDense(layers.Layer):
    # to replace standard layers.Dence()
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
        #  Layer.add_variable is deprecated
        # self.kernel = self.add_variable('w', [inp_dim, outp_dim])
        # self.bias = self.add_variable('b', [outp_dim])   
        self.kernel = self.add_weight('w', [inp_dim, outp_dim])


    def call(self, inputs, training=None):
        x = inputs @ self.kernel
        return x

In [9]:
class MyNetwork(keras.Model):
    
    def __init__(self):
        super(MyNetwork, self).__init__()

        self.fc1 = MyDense(32*32*3, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)


    def call(self, inputs, training=None):
        # forward
        """
        inputs: [b, 32, 32, 3]
        training

        """
        x = tf.reshape(inputs, [-1, 32*32*3])

        # (b, 32*32*3)   [b, 256]
        x = self.fc1(x)
        x = tf.nn.relu(x)

        # [b, 256]  [b, 128]
        x = self.fc2(x)
        x = tf.nn.relu(x)

        # [b, 128]  [b, 64]
        x = self.fc3(x)
        x = tf.nn.relu(x)

        # [b, 64]  [b, 32]
        x = self.fc4(x)
        x = tf.nn.relu(x)

        # [b, 32]  [b, 10]
        x = self.fc5(x)

        return x


In [10]:
netWork = MyNetwork()
netWork.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['acc'])

In [11]:
netWork.fit(train_db, epochs=15, validation_data=test_db, validation_freq=1)

Epoch 1/15
391/391 [==============================] - 2s 5ms/step - loss: 1.7305 - acc: 0.3886 - val_loss: 1.5806 - val_acc: 0.4407
Epoch 2/15
391/391 [==============================] - 2s 5ms/step - loss: 1.5115 - acc: 0.4692 - val_loss: 1.5136 - val_acc: 0.4609
Epoch 3/15
391/391 [==============================] - 2s 5ms/step - loss: 1.4109 - acc: 0.5010 - val_loss: 1.4362 - val_acc: 0.4976
Epoch 4/15
391/391 [==============================] - 2s 5ms/step - loss: 1.3271 - acc: 0.5322 - val_loss: 1.4184 - val_acc: 0.5039
Epoch 5/15
391/391 [==============================] - 2s 5ms/step - loss: 1.2628 - acc: 0.5563 - val_loss: 1.4213 - val_acc: 0.5068
Epoch 6/15
391/391 [==============================] - 2s 5ms/step - loss: 1.1947 - acc: 0.5794 - val_loss: 1.4190 - val_acc: 0.5082
Epoch 7/15
391/391 [==============================] - 2s 5ms/step - loss: 1.1387 - acc: 0.5995 - val_loss: 1.4074 - val_acc: 0.5229
Epoch 8/15
391/391 [==============================] - 2s 5ms/step - loss: 1.

In [12]:
netWork.evaluate(test_db)
if not os.path.exists('ckpt'):
    os.mkdir('ckpt')
netWork.save_weights('ckpt/weights.ckpt')
print("saved to ckpt/weights.ckpt")
del netWork

79/79 [==============================] - 0s 3ms/step - loss: 1.6604 - acc: 0.5206
saved to ckpt/weights.ckpt


In [13]:
netWork = MyNetwork()
netWork.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['acc'])
netWork.load_weights('ckpt/weights.ckpt')
print("load ckpt/weights.ckpt")
netWork.evaluate(test_db)

load ckpt/weights.ckpt
79/79 [==============================] - 0s 3ms/step - loss: 1.6604 - acc: 0.5206


[1.6604353189468384, 0.5206000208854675]